Model Training (Random Forest)

In [ ]:
# Import Modules
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import MinMaxScaler
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Load the training dataset
train_data = pd.read_csv(r'D:\Documents\ITB\S2\WF5011 - Design of Experiment and Data Analytics\Project-02-Data\Project 02-Data\train.csv')

In [ ]:
# Split features (X) and target (y)
X = train_data.drop('price_range', axis=1)
y = train_data['price_range']

In [ ]:
# Normalize the features using Min-Max Scaling
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize and train the Random Forest Classifier
rf_model = RandomForestClassifier(
    n_estimators=100,  # Number of trees
    max_depth=None,    # Allow trees to grow fully
    random_state=42    # For reproducibility
)
rf_model.fit(X_train, y_train)

In [ ]:
# Make predictions on the validation set
y_pred = rf_model.predict(X_val)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='weighted')  # Weighted F1 score
print(f"Validation Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

In [ ]:
# Confusion Matrix Heatmap
cm = confusion_matrix(y_val, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # Normalize by row

plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=[0, 1, 2, 3], yticklabels=[0, 1, 2, 3])
plt.title('Confusion Matrix Heatmap (Normalized)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# Feature Importance Plot
feature_importances = rf_model.feature_importances_
feature_names = X.columns
sorted_idx = np.argsort(feature_importances)[::-1]

plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importances)), feature_importances[sorted_idx], align='center')
plt.xticks(range(len(feature_importances)), feature_names[sorted_idx], rotation=90)
plt.title('Feature Importance Plot')
plt.xlabel('Features')
plt.ylabel('Importance Score')
plt.show()

In [ ]:
# Save the model and the scaler for future use
joblib.dump(rf_model, 'random_forest_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("Model and scaler saved successfully!")

Model Deployment on Test Data

In [ ]:
# Load test dataset
test_data = pd.read_csv(r'D:\Documents\ITB\S2\WF5011 - Design of Experiment and Data Analytics\Project-02-Data\Project 02-Data\test.csv')

In [ ]:
# Load the saved model and scaler
rf_model = joblib.load(r'D:\Documents\ITB\S2\WF5011 - Design of Experiment and Data Analytics\random_forest_model.pkl')
scaler = joblib.load(r'D:\Documents\ITB\S2\WF5011 - Design of Experiment and Data Analytics\scaler.pkl')

In [ ]:
# Normalize the test dataset
test_features = test_data.drop('id', axis=1)  # Exclude non-feature columns like 'id'
test_features_normalized = scaler.transform(test_features)

In [ ]:
# Make predictions
predictions = rf_model.predict(test_features_normalized)

In [ ]:
# Add predictions to the test dataset
test_data['price_range'] = predictions

In [ ]:
# Save predictions to a CSV file
test_data.to_csv('test_predictions.csv', index=False)
print("Predictions saved to 'test_predictions.csv'.")

In [ ]:
# Test model with manual input
user_input = pd.DataFrame({
    'battery_power': 1900,
    'blue': 1,
    'clock_speed': 0.5,
    'dual_sim': 1,
    'fc': 0,
    'four_g': 1,
    'int_memory': 55,
    'm_dep': 0.9,
    'mobile_wt': 171,
    'n_cores': 1,
    'pc': 1,
    'px_height': 934,
    'px_width': 1241
    'ram': 3917,
    'sc_h': 15,
    'sc_w': 5,
    'talk_time': 18,
    'three_g': 1,
    'touch_screen': 0,
    'wifi': 1
    })

# Normalize the features using the saved scaler
user_input_normalized = scaler.transform(user_input)

# Predict the price range using the saved model
prediction = rf_model.predict(user_input_normalized)

# Output the predicted price range
print(f"\nPredicted Price Range: {prediction[0]}")